# RAD-seq assembly with ipyrad

In [56]:
import ipyrad as ip
import pandas as pd
import glob

### Sample metadata

#### CSV file mapping barcodes combinations (names in already demux'd fastq files) to accession IDs

In [59]:
bcdf = pd.read_csv("../metadata/barcode_map.csv", sep="\t")
bcdf.head(10)

,barcode,accession
0,TAAGGCGA-TAGATCGC,1530
1,CGTACTAG-TAGATCGC,1531
2,AGGCAGAA-TAGATCGC,1535
3,TCCTGAGC-TAGATCGC,1540
4,GGACTCCT-TAGATCGC,1541
5,TAGGCATG-TAGATCGC,1542
6,CTCTCTAC-TAGATCGC,1545
7,CAGAGAGG-TAGATCGC,1548
8,GCTACGCT-TAGATCGC,1549
9,CGAGGCTG-TAGATCGC,1550


#### CSV file with accession IDs, species names, and sample info
Here we want to use the `specie` label which is the new updated name (`oldname` is original description).

In [60]:
df = pd.read_csv("../metadata/pablo-cacti-relabeled-re.csv")
df.head(10)

,accession,Genus,specie,subspecie,Localidad,Lat,Long,oldname
0,1075,Eriosyce,litoralis,NaN,TotoralilloIVR,-30.068180,-71.375040,litoralis
1,1083,Eriosyce,litoralis,NaN,TotoralilloIVR,-30.069180,-71.375780,litoralis
2,1093,Eriosyce,clavata,NaN,Qda.Manqueza,-29.913730,-70.943290,clavata
3,1288,Eriosyce,subgibbosa,NaN,PeninsuladeHualpenVIIIR,-36.757410,-73.176340,subgibbosa
4,1290,Eriosyce,subgibbosa,NaN,PeninsuladeHualpenVIIIR,-36.757410,-73.176340,subgibbosa
5,1332,Eriosyce,castanea,NaN,CerroLaLajuela(StaCruz),-34.664130,-71.416970,castanea
6,1333,Eriosyce,castanea,NaN,CerroLaLajuela(StaCruz),-34.664150,-71.416960,castanea
7,1349,Eriosyce,litoralis,NaN,Pichidanguii,-32.156120,-71.528270,subgibbosa
8,1350,Eriosyce,litoralis,NaN,Pichidanguii,-32.156000,-71.528190,subgibbosa
9,1351,Eriosyce,litoralis,NaN,Pichidanguii,-32.156112,-71.528243,subgibbosa


### Data Assembly
Select the already demultiplexed data from fastq files and assemble with no restriction overhang detection, and using the Saguaro reference genome.

In [74]:
data = ip.Assembly("Eriosyce-2021")
data.params.project_dir = "../assembly"
data.params.sorted_fastq_path = "../RAWDATA/[!Undet]*.gz"
data.params.assembly_method = "reference"
data.params.reference_sequence = "../reference/saguaro_v1.3.fna"
data.params.restriction_overhang = ("", "")
data.params.filter_adapters = 2

# nextrad adapter sequence
data.params.filter_min_trim_len = 50
data.params.filter_adapters = 3
data.params.mindepth_majrule = 2
data.params

New Assembly: Eriosyce-2021


0   assembly_name               Eriosyce-2021                                
1   project_dir                 /pinky/deren/pablo-cacti/assembly            
2   raw_fastq_path                                                           
3   barcodes_path                                                            
4   sorted_fastq_path           /pinky/deren/pablo-cacti/RAWDATA/[!Undet]*.gz
5   assembly_method             reference                                    
6   reference_sequence          /pinky/deren/pablo-cacti/reference/saguaro_v1.3.fna
7   datatype                    rad                                          
8   restriction_overhang        ('', '')                                     
9   max_low_qual_bases          5                                            
10  phred_Qscore_offset         33                                           
11  mindepth_statistical        6                                            
12  mindepth_majrule            2                         

In [75]:
adapter = "CTGTCTCTTATACACATCTCCGAGCCCACGAGACCTCTCTACATCTCGTATGCCGTCTTCTGCTTGA"
data.hackersonly.p3_adapters_extra.append(adapter[:15])
data.hackersonly.p5_adapters_extra.append(adapter[:15])
data.hackersonly

0   random_seed                 42                                           
1   max_fragment_length         50                                           
2   max_inner_mate_distance     500                                          
3   p5_adapter                  AGATCGGAAGAGC                                
4   p3_adapter                  AGATCGGAAGAGC                                
5   p3_adapters_extra           ['CTGTCTCTTATACAC']                          
6   p5_adapters_extra           ['CTGTCTCTTATACAC']                          
7   query_cov                   None                                         
8   bwa_args                                                                 
9   demultiplex_on_i7_tags      False                                        
10  declone_PCR_duplicates      False                                        
11  merge_technical_replicates  True                                         
12  exclude_reference           True                            

In [76]:
data.ipcluster['cores'] = 10
data.run("1", auto=True, force=True)

Parallel connection | pinky: 10 cores
[####################] 100% 0:05:38 | loading reads        | s1 |
Parallel connection closed.


### Rename samples and join technical replicates
The samples are currently labeled by barcode and I want them instead to be labeled by accession.

In [127]:
# example sample name
list(data.samples.keys())[0]

'TCCTGAGC-GTAAGGAG_S52_L003001'

In [128]:
# make dict mapping barcodes to accession IDs
barcode_to_accession = {}

for sample in data.samples:
    bc = sample.split("_")[0]
    acc = bcdf.loc[(bcdf.barcode == bc), "accession"].iloc[0]
    barcode_to_accession[sample] = acc

In [129]:
# create copy of assembly that will use NEW NAMES
fdata = ip.merge("Eriosyce-ref-2021", [data], rename_dict=barcode_to_accession)

New Assembly: Eriosyce-ref-2021


In [130]:
# show that .stats now uses new names
fdata.stats.head()

,state,reads_raw
1075,1,2673356
1083,1,2212128
1093,1,3838041
1288,1,1583203
1290,1,3033586


### Assemble all samples 
Create a single min4 assembly and we will use the ipyrad-analysis tools to subselect loci for further analyses.

In [131]:
# assemble the dataset with all samples for steps 2-7
fdata.ipcluster['cores'] = 10
fdata.ipcluster['threads'] = 2
fdata.run('234567', force=True, auto=True)

Parallel connection | pinky: 10 cores
[####################] 100% 1:23:08 | processing reads     | s2 |
[####################] 100% 0:00:01 | indexing reference   | s3 |
[####################] 100% 0:07:30 | dereplicating        | s3 |
[#################   ]  89% 2:20:37 | mapping reads        | s3 |

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[################### ]  96% 2:31:19 | mapping reads        | s3 |

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[###############     ]  77% 0:09:38 | building clusters    | s3 |

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[####                ]  20% 0:18:50 | consens calling      | s5 |

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[####################] 100% 0:50:03 | consens calling      | s5 |
[####################] 100% 0:00:54 | indexing alleles     | s5 |
[####################] 100% 0:01:11 | concatenating bams   | s6 |
[####################] 100% 0:00:13 | fetching regions     | s6 |
[####################] 100% 0:00:46 | building database    | s6 |
[####################] 100% 0:00:53 | applying filters     | s7 |
[####################] 100% 0:08:46 | building arrays      | s7 |
[#############       ]  66% 0:03:12 | writing conversions  | s7 |

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Preliminary phylogenetic analysis
Ensure that samples group on phylogeny as expected. Once confirmed, we will then exclude the samples that are not relevant to this study, and re-run step 7 to obtain a final assembly and database with all loci for the target set of samples. 


In [135]:
import ipyrad.analysis as ipa
import toytree

In [140]:
# filter and select an alignment with max 50% missing data
wex = ipa.window_extracter(
    name='Eriosyce-ref-2021',
    workdir='../analysis-window_extracter/',
    data="/pinky/deren/pablo-cacti/assembly/Eriosyce-ref-2021_outfiles/Eriosyce-ref-2021.seqs.hdf5",
    scaffold_idxs=range(5000),
    mincov=0.5,
    rmincov=0.05,
)

# show alignment size
display(wex.stats)

# run extracter to write phylip file output
wex.run(force=True)

,scaffold,start,end,sites,snps,missing,samples
0,concatenated,0,319947,319947,20961,0.291,97


Wrote data to /pinky/deren/pablo-cacti/analysis-window_extracter/Eriosyce-ref-2021.phy


In [147]:
# run raxml on the new alignment phy file
rax = ipa.raxml(
    name="Erioscye-ref-2021",
    workdir="../analysis-raxml",
    data="../analysis-window_extracter/Eriosyce-ref-2021.phy",
    T=11,
    m="GTRCAT"
)

# print the raxml command
print(rax.command)

# run raxml inference
rax.run(force=True)

/home/deren/miniconda3/envs/ipy/bin/raxmlHPC-PTHREADS-AVX2 -f a -T 11 -m GTRCAT -n Erioscye-ref-2021 -w /pinky/deren/pablo-cacti/analysis-raxml -s /pinky/deren/pablo-cacti/analysis-window_extracter/Eriosyce-ref-2021.phy -p 54321 -N 100 -x 12345
job Erioscye-ref-2021 finished successfully


In [170]:
# plot the tree result
tre = toytree.tree("../analysis-raxml/RAxML_bipartitions.Erioscye-ref-2021")

# get mapping of accessions to sample names
namedict = dict(zip(df.accession, df.specie + "_" + df.accession))

# add extra names
namedict['1530x'] = "undet_1530x"
namedict['reference'] = "Saguaro"

# root on suspected outgroup (reference)
tre = tre.root("reference")

# draw tree with sample names
tre.draw(
    tip_labels=[namedict[i] for i in tre.get_tip_labels()],
);

<svg class="toyplot-canvas-Canvas" height="1000.0px" id="ta8d0835910764cf781b091195b1e08d0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 395.0 1000.0" width="395.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> chilensis_1550 chilensis_1549 chilensis_1548 chilensis_1540 chilensis_1542 chilensis_1535 chilensis-albidiflora_1524 chilensis-albidiflora_1523 undet_1530x chilensis_1530 chilensis-albidiflora_1509 chilensis-albidiflora_1508 chilensis-albidiflora_1580 chilensis-albidiflora_1583 chilensis-albidiflora_1582 chilensis-albidiflora_1511 chilensis-albidiflora_1512 chilensis_1562 chilensis_1556 chilensis-albidiflora_1516 chilensis-albidiflora_1521 chilensis_1560 chilensis_1594 chilensis_1593 chilensis_1531 chilensis_1570 chilensis-albidiflora_1518 chilensis_1541 chilensis-albidiflora_1585 chilensis-albidiflora_1584 chilensis-albidiflora_1510 chilensis_1545 chilensis-albidiflora_1522 hybrid_HPG2 hybrid_HPG1 hybrid_HPG4 hybrid_SXM2 hybrid_SXM1 litoralis_1379 litoralis_1387 chilensis_1597 litoralis_1368 litoralis_1369 litoralis_1358 litoralis_1359 litoralis_1351 litoralis_1355 litoralis_1349 castanea_1332 castanea_1333 litoralis_1354 litoralis_1362 litoralis_1353 litoralis_1377 litoralis_1375 litoralis_1374 litoralis_1372 litoralis_1383 litoralis_1350 litoralis_1364 litoralis_1356 chilensis_1596 litoralis_1385 litoralis_1384 litoralis_1371 senilis_BV185 senilis_BV152 litoralis_1075 litoralis_1352 litoralis_1083 clavata_1093 subgibbosa_1288 mutabilis_1591 mutabilis_1527 mutabilis_1536 mutabilis_1525 mutabilis_1546 mutabilis_1538 mutabilis_1539 mutabilis_1588 mutabilis_1590 mutabilis_1589 mutabilis_1592 clavata_BV85 mutabilis_1567 mutabilis_1553 mutabilis_1554 mutabilis_1587 mutabilis_1547 armata_9366 aspillagae_1488 subgibbosa_1290 curvispina-aconcaguensis_HV55 curvispina-aconcaguensis_HV54 marksiana_BV190 aurata_BV113 Saguaro

### Justification for sampling

The samples 1530 and 1530x have strange long branch lengths, and one is of uncertain determination, so we will leave both samples out. The clavata samples BV85 and 1093 were collected close to each other, and identified as the same, but group quite distant on the tree. I suspect 1093 is the true position, and BV85 may be a mislabeling error. Because this species is not important to this study we exclude both samples.

The hybrid samples will be kept in the assembly, but will be dropped from some analyses later.

The list below will be used in other notebooks to subsample taxa prior to analyses. 

In [ ]:
# these are dropped from ALL due to potential labeling error
#undet_1530x
#chilensis_1530
#clavata_BV85
#clavata_1093

# these will be dropped from some b/c they are not focal spp.
# hybrid_HPG2
# hybrid_HPG1
# hybrid_HPG4
# hybrid_SXM2
# hybrid_SXM1
# castanea_1332
# castanea_1333
# senilis_BV185
# senilis_BV152
# subgibbosa_1288
# subgibbosa_1290

# low amounts of data
# 1351
# 1352
# 1353
# 1541
# 1594

In [182]:
fdata.stats_dfs.s7_samples[fdata.stats_dfs.s7_samples.sample_coverage < 5000]

,sample_coverage
1351,2565
1352,1693
1353,3418
1530,1920
1530x,1164
1541,2370
1594,4987


In [183]:
DROP = [
    '1530',
    '1530x',
    'BV85',
    '1093',
    'HPG1',
    'HPG2',
    'HPG4',
    'SXM1',
    'SXM2',
    'SXM3',
    '1332',
    '1333',
    'BV185',
    'BV152',
    '1288',
    '1290',
    'reference',
    '1351',
    '1352',
    '1353',
    '1541',
    '1594',
]
len(DROP)

22